Import the required libraries

In [1]:
import pandas as pd

from bs4 import BeautifulSoup
import requests
import csv

import re
import time
import datetime

import smtplib

Create a csv file to record the daily price

In [2]:
header = ['Title','Price','Date']

with open('AmazonProductsDataset.csv','w',newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)

Connect to the website and pull in data

In [3]:
def check_price():
    #url of the item you need to monitor    
    url = "https://www.amazon.com/Star-Trek-Generation-Operations-Uniform/dp/B08P2J699Y/ref=sr_1_2?crid=S8HSC5AUKWNZ&keywords=data+tshirt&qid=1648462834&sprefix=data+tshir%2Caps%2C403&sr=8-2"

    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36", 
            "Accept-Encoding": "gzip, deflate, br", 
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
            "DNT":"1",
            "Connection":"close", 
            "Upgrade-Insecure-Requests":"1"}

    page = requests.get(url, headers=headers)

    #parses the page content and displayed as special formatting
    soup = BeautifulSoup(page.content, "html.parser")
    pretty_soup = soup.prettify()

    #Extract the title and price of the product
    title = soup.find(id='productTitle').get_text().strip()
    #print(title)

    price = float(re.findall('\d+.\d+', soup.find(id="corePrice_desktop").get_text())[0])
    
    #email if the price drops
    if(price<=15):
            send_email()
    #print(price)

    #get today's date
    date = datetime.date.today()
    #print(date)

    #reviews = re.findall('\d+', soup.find(id='averageCustomerReviews_feature_div').get_text())  #try to extract the rating as 4.4 instead.
    #print(reviews)

    data = [title,price,date]

    with open('AmazonProductsDataset.csv','a+',newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)


In [4]:
#View the csv File

check_price()

df = pd.read_csv('AmazonProductsDataset.csv')

print(df)

                                               Title  Price        Date
0  Star Trek: The Next Generation Operations Unif...  29.95  2022-04-03


Email it to someone when the price drops

In [ ]:


def send_email():
    server = smtplib.SMTP_SSL('smtp.gmail.com',465)
    server.ehlo()
    #server.starttls()
    #server.ehlo()
    server.login('sender***email@gmail.com','pass*******word')
    
    subject = "The Shirt you want is below $15! Now is your chance to buy!"
    body = "Hi Lotus Shan, This is the moment we have been waiting for. Now is your chance to pick up the shirt of your dreams. Don't mess it up! Link here: https://www.amazon.com/Star-Trek-Generation-Operations-Uniform/dp/B08P2J699Y/ref=sr_1_2?crid=S8HSC5AUKWNZ&keywords=data+tshirt&qid=1648462834&sprefix=data+tshir%2Caps%2C403&sr=8-2"
   
    msg = f"Subject: {subject}\n\n{body}"
    
    server.sendmail('receiver***email@gmail.com',msg)

Run the above code in certain time period to keep a track on the product details

In [ ]:
# Runs check_price in certain time interval and inputs data into your CSV

while(True):
    check_price()
    time.sleep(3600) # 3600 sec = 1 hour
